<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Evaluating a Multi-Object Tracking Model on MOT Challenge

This notebook is for evaluating [FairMOT](https://github.com/ifzhang/FairMOT) on the [MOT Challenge dataset](https://motchallenge.net/), one of the most common benchmarking datasets for measuring multi-object tracking performance on pedestrian data. The collection includes MOT15, MOT16/17, and MOT 19/20. These datasets contain various video sequences, each with different tracking difficulty levels and ground-truth annotations. Detections are also provided for optional use.

The goal of this notebook is to re-produce published results on the MOT challenge using the state-of-the-art FairMOT approach.

## Initialization

In [1]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import os.path as osp
import sys
import time

from urllib.parse import urljoin
import torch
import torchvision

sys.path.append("../../")
from utils_cv.common.data import data_path, download, unzip_url
from utils_cv.common.gpu import which_processor, is_windows
from utils_cv.tracking.data import Urls
from utils_cv.tracking.dataset import TrackingDataset
from utils_cv.tracking.model import TrackingLearner

# Change matplotlib backend so that plots are shown for windows
if is_windows():
    plt.switch_backend("TkAgg")

print(f"TorchVision: {torchvision.__version__}")
which_processor()

TorchVision: 0.4.0a0+6b959ee
Torch is using GPU: Tesla K80


This shows your machine's GPUs (if it has any) and the computing device `torch/torchvision` is using.

Next, set some model runtime parameters. We evaluate the FairMOT model on MOT17.

In [3]:
CONF_THRES = 0.4
TRACK_BUFFER = 30
IM_SIZE = (1080, 1920)

# Downloaded MOT Challendage data path
MOT_ROOT_PATH = "../../data/"
RESULT_ROOT = "./results"
EXP_NAME = "MOT_val_all_dla34"

BASELINE_MODEL = "./models/all_dla34.pth"
MOTCHALLENGE_BASE_URL = "https://motchallenge.net/data/"

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using torch device: {device}")

Using torch device: cuda


## Pretrained Model Inference

Download [MOT17](https://motchallenge.net/data/MOT17.zip) data to `MOT_SAVED_PATH`. The MOT17 dataset is around 5GB. Note that it may take some time to download.

Since MOT17 provides ground truths in the training data, we evaluate FairMOT on it by [py-motmetrics](https://github.com/cheind/py-motmetrics). For evaluating on testing dataset, see 'Evaluate on Test Set' section.

In [4]:
mot_path = urljoin(MOTCHALLENGE_BASE_URL, "MOT17.zip")
MOT_TRAIN_PATH = osp.join(MOT_ROOT_PATH, "MOT17", "train")
MOT_TEST_PATH = osp.join(MOT_ROOT_PATH, "MOT17", "test")
# seqs_str:  various video sequences subfolder names under MOT challenge data
train_seqs_str = """MOT17-02-SDP
                    MOT17-04-SDP
                    MOT17-05-SDP
                    MOT17-09-SDP
                    MOT17-10-SDP
                    MOT17-11-SDP
                    MOT17-13-SDP"""
test_seqs_str = """MOT17-01-SDP
                   MOT17-03-SDP
                   MOT17-06-SDP
                   MOT17-07-SDP
                   MOT17-08-SDP
                   MOT17-12-SDP
                   MOT17-14-SDP"""

unzip_url(mot_path, dest=MOT_SAVED_PATH, exist_ok=True)
print(f"Training data saved to {MOT_TRAIN_PATH}")
print(f"Test data saved to {MOT_TEST_PATH}")

Training data saved to ../../data/MOT17/train
Test data saved to ../../data/MOT17/test


Initialize and load the model. We use the pre-trained baseline FairMOT model - `all_dla34.pth`, which can be downloaded [here](https://drive.google.com/file/d/1udpOPum8fJdoEQm6n0jsIgMMViOMFinu/view). Please upload and save `all_dla34.pth` to `BASELINE_MODEL` path.

In [5]:
tracker = TrackingLearner(None, BASELINE_MODEL)

## Evaluate on Training Set

We use training dataset from MOT17 to evaluate with the ground truth.

In [6]:
seqs = [seq.strip() for seq in train_seqs_str.split()]

strsummary = tracker.eval_mot(
    conf_thres=CONF_THRES,
    track_buffer=TRACK_BUFFER,
    im_size=IM_SIZE,
    data_root=MOT_TRAIN_PATH,
    seqs=seqs,
    result_root=RESULT_ROOT,
    exp_name=EXP_NAME,
)
print(strsummary)

Saved tracking results to ./results/MOT_val_all_dla34/MOT17-02-SDP.txt
Evaluate seq: MOT17-02-SDP
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-04-SDP.txt
Evaluate seq: MOT17-04-SDP
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-05-SDP.txt
Evaluate seq: MOT17-05-SDP
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-09-SDP.txt
Evaluate seq: MOT17-09-SDP
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-10-SDP.txt
Evaluate seq: MOT17-10-SDP
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-11-SDP.txt
Evaluate seq: MOT17-11-SDP
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-13-SDP.txt
Evaluate seq: MOT17-13-SDP
              IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML    FP    FN IDs    FM  MOTA  MOTP IDt IDa IDm
MOT17-02-SDP 60.1% 68.9% 53.3% 72.3% 93.4%  62  28  25  9   950  5148 243   628 65.9% 0.198 138  49  16
MOT17-04-SDP 83.2% 85.1% 81.2% 86.3% 90.4%  83  51  21 11  4348  6526  32   219 77.1% 0.172   6  20   1
MO

## Evaluate on Test Set

The reported evaluation results from [FairMOT paper](https://arxiv.org/abs/2004.01888) with test set are as follows:

| Dataset | MOTA   | IDF1 | IDS | MT | ML | FPS |
|------|------|------|------|------|------|------|
|   MOT16  | 68.7| 70.4| 953| 39.5%| 19.0%| 25.9|
|   MOT17  | 67.5| 69.8| 2868| 37.7%| 20.8%| 25.9|

For evaluating on testing dataset, you can get the txt prediction results in this section and submit to the [MOT Challenge](https://motchallenge.net/) evaluation server to obtain the results. You can get the SOTA results 68.5 MOTA on MOT17 test set using the baseline model 'all_dla34.pth' from the [MOT17 evaluation server](https://motchallenge.net/results/MOT17/?det=Private).
<img src="media/mot_results.PNG" style="width: 737.5px;height: 365px"/>


In [7]:
seqs = [seq.strip() for seq in test_seqs_str.split()]

tracker.eval_mot(
    conf_thres=CONF_THRES,
    track_buffer=TRACK_BUFFER,
    im_size=IM_SIZE,
    data_root=MOT_TEST_PATH,
    seqs=seqs,
    result_root=RESULT_ROOT,
    exp_name=EXP_NAME,
    if_test_data=True,
)

Saved tracking results to ./results/MOT_val_all_dla34/MOT17-01-SDP.txt
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-03-SDP.txt
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-06-SDP.txt
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-07-SDP.txt
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-08-SDP.txt
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-12-SDP.txt
Saved tracking results to ./results/MOT_val_all_dla34/MOT17-14-SDP.txt
